In [1]:
import os
import pyodbc 

import pandas             as pd
import numpy              as np
from sklearn.cluster      import KMeans

import matplotlib.pyplot  as plt

# Comment this if the data visualisations doesn't work
%matplotlib inline
plt.style.use('bmh')

In [2]:
path = 'C:/Users/banarn/'
os.chdir(path)

## Connection string
To make connection with DataBase (SQL, Access, Excel, etc.) required library PYODBC

More good samples how to use can finde in:
    [Wiki](https://github.com/mkleehammer/pyodbc/wiki)


In [3]:
# required pyodbc

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=Z:\Swedwood KR\Department Production\Technologist\BOM\DB\Inventories_v3_be.accdb;'
    r'UID=user;'
    r'PWD=;'
    )
cnxn = pyodbc.connect(conn_str)

SQL query string to take data for work

In [4]:
sql = """   
    SELECT 
        ITEMCHAR.*,
        LEFT(ITEM_ID, inStr(ITEM_ID, '-')+2) +
            COLLORS.COLINX +
            RIGHT(ITEM_ID, LEN(ITEM_ID)-(inStr(ITEM_ID, '-')+3)) as item,
        LEFT(ITEM_ID, inStr(ITEM_ID, '-')-1) as br   
    FROM ITEMCHAR, COLLORS
    WHERE 
        ITEMCHAR.COLOR_INDEX = COLLORS.UNKNOWN AND 
        WC in (12020, 12010)        
    """
data = pandas.read_sql(sql,cnxn)

NameError: name 'pandas' is not defined

methods for clustering:
    
* [HDBSCAN](http://hdbscan.readthedocs.io/en/latest/index.html)
* DBSCAN
* K Narest

In [ ]:
iris = pandas.read_csv("./input/Iris.csv")

In [ ]:
iris.head()

In [ ]:
iris.info()

In [ ]:
iris_matrix = pandas.DataFrame.as_matrix(iris[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']])
kmeans = KMeans(n_clusters=3)
kmeans.fit(iris_matrix)